# Ingest Twitter Feed Data and Sentiments into iguazio Stream & Time-Series DB 

## Initialization 
install packages and set environment variables.<br>
need to fill the following environment variables with real credentials.

```
    V3IO_PASSWORD = <V3IO-Password>
    V3IO_USER = <V3IO-Username>
    V3IO_ADDRESS = <address of V3IO API end point>
    
    # Twitter credentials
    app_key = <..>
    app_secret = <..>
    oauth_token = <..>
    oauth_token_secret = <..>

```
> note: in nuclio environment variables and build dependencies are specified in the configuration tab, for notebooks they can be initialized from a file as implemented below.

In [ ]:
# nuclio: ignore
!pip install textblob
!pip install twython

### initialize nuclio emulation (this section will be ignored by nuclio nbconvert)

In [5]:
# nuclio: ignore
from nuclio import Context, Event
from v3io import env_fromfile
env_fromfile('tweet_env.txt')
context = Context()

### Twitter stream handling class

In [9]:
from twython import TwythonStreamer
import json
import re
import v3io
import os
from textblob import TextBlob
import v3io_frames as v3f
import pandas as pd

oauth = {
    'app_key' : os.getenv('app_key'),
    'app_secret' : os.getenv('app_secret'),  
    'oauth_token' : os.getenv('oauth_token'), 
    'oauth_token_secret' : os.getenv('oauth_token_secret'),
}
lastText = ''

# initialize iguazio v3io APIs
v3 = v3io.v3io(os.getenv('V3IO_ADDRESS'),os.getenv('V3IO_USER'),os.getenv('V3IO_PASSWORD'), 'bigdata')
client = v3f.Client('v3io-framesd:8081', password=os.getenv('V3IO_PASSWORD'))

# Twitter stream handler  
class MyStreamer(TwythonStreamer):
    def __init__(self, context, name, **kw):
        self.name = name
        self.context = context
        TwythonStreamer.__init__(self, **kw)
        
    def start(self, cb, limit=10, **kw):
        self.cb = cb
        self.limit = limit
        self.statuses.filter(**kw)
        
    def on_success(self, data):
        if 'text' in data:
            record = {'text': data['text'], 
                      'user': '@'+data['user']['screen_name'],
                      'id': data['id'],
                      'created_at':data['created_at'],
                     }
            self.context.last_message = record
            if self.cb:
                self.cb(self.context, self.name, record)
                
        self.limit -= 1 
        if self.limit <= 0 :
            self.disconnect()

    def on_error(self, status_code, data):
        self.context.logger.error_with('Error in stream', status_code=status_code)

def process_event(context, name, record):
    clean = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", record['text']).split())
        
    # enrich the record with natural language metadata
    blob = TextBlob(clean)
    record['polarity'] = blob.sentiment.polarity
    record['subjectivity'] = blob.sentiment.subjectivity

    # Write the record into a iguazio straem
    context.logger.debug_with('writing data to Stream', record=record)
    resp = v3.putrecords('stock_stream', [json.dumps(record)])
        
    # Write data to iguazio Time-Series DB
    df = pd.DataFrame(index=[[pd.to_datetime(record['created_at'])],['GOOG']], columns=['sentiment'])
    df['sentiment'] = [float(blob.sentiment.polarity)]
    df.index.names=['time','symbol']
    client.write(backend='tsdb', table='stock_metrics',dfs=df)

## Nuclio service initialization (init_context) and event handler implementation

In [10]:
import threading

def start_listener(context):
    stream = MyStreamer(context, 'GOOG', **oauth)
    stream.start(process_event, 200, track='@Google', lang='en')

def handler(context, event):
    return json.dumps(context.last_message)

def init_context(context):
    context.last_message = {}
    t = threading.Thread(target=start_listener, args=(context,))
    t.start()

## Function invocation
the following section simulates nuclio function initialization and invocation

In [11]:
# nuclio: ignore
init_context(context)
event = Event(body='')

2018-12-17 19:53:20,524 nuclio (D) writing data to Stream {"record": {"text": "Hey you awesome people at @PeelPublicWorks @regionofpeel \nDo you know that if an URL is too long then you can short\u2026 https://t.co/e9gfGEBKX9", "user": "@NileshR_Shah", "id": 1074754410532741122, "created_at": "Mon Dec 17 19:53:14 +0000 2018", "polarity": 0.31666666666666665, "subjectivity": 0.5666666666666667}}
2018-12-17 19:53:20,550 nuclio (D) writing data to Stream {"record": {"text": "Hey you awesome people at @PeelPublicWorks @regionofpeel \nDo you know that if an URL is too long then you can short\u2026 https://t.co/e9gfGEBKX9", "user": "@NileshR_Shah", "id": 1074754410532741122, "created_at": "Mon Dec 17 19:53:14 +0000 2018", "polarity": 0.31666666666666665, "subjectivity": 0.5666666666666667}}
2018-12-17 19:53:29,024 nuclio (D) writing data to Stream {"record": {"text": "\u2018 CCIA (@Google\u2069 etc)..acknowledges that #Art13 will \u201cbenefit rightholders.\u201d ...[but] their messaging in 

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/conda/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py", line 294, in recv_into
    return self.connection.recv_into(*args, **kwargs)
  File "/conda/lib/python3.6/site-packages/OpenSSL/SSL.py", line 1814, in recv_into
    self._raise_ssl_error(self._ssl, result)
  File "/conda/lib/python3.6/site-packages/OpenSSL/SSL.py", line 1614, in _raise_ssl_error
    raise WantReadError()
OpenSSL.SSL.WantReadError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/conda/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py", line 294, in recv_into
    return self.connection.recv_into(*args, **kwargs)
  File "/conda/lib/python3.6/site-packages/OpenSSL/SSL.py", line 1814, in recv_into
    self._raise_ssl_error(self._ssl, result)
  File "/conda/lib/python3.6/site-packages/OpenSSL/SSL.py", line 1631, in _raise_ssl_error
    raise SysCallError(errno, errorc

2018-12-17 20:14:33,858 nuclio (D) writing data to Stream {"record": {"text": "@google how can services be just killed like this? Orkut with updates would of have survived. As well as Picasa. No\u2026 https://t.co/JOqbHBzpYO", "user": "@ufopeace", "id": 1074759755636400128, "created_at": "Mon Dec 17 20:14:28 +0000 2018", "polarity": -0.2, "subjectivity": 0.0}}
2018-12-17 20:14:39,318 nuclio (D) writing data to Stream {"record": {"text": "RT @loscarnaless: Hola (#TripAdvisor @TripAdvisor - @TripAdvisorES):\n\nAqu\u00ed le dejo un v\u00eddeo a @Google para que pueda indexarlo y todo el mu\u2026", "user": "@Maternitat_SEDR", "id": 1074759779044724738, "created_at": "Mon Dec 17 20:14:34 +0000 2018", "polarity": 0.0, "subjectivity": 0.0}}
2018-12-17 20:14:41,487 nuclio (D) writing data to Stream {"record": {"text": "@1NewsNZ @jacindaardern @Google @jacktame @Breakfaston1 A great week of deflection whilst winnie signs the UN Migration pact!\n#TreasonousPM", "user": "@ClawDove", "id": 1074759